In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

In [3]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [4]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_teddynote.tools import GoogleNews
from langchain.agents import tool
from typing import List, Dict
from bs4 import BeautifulSoup
import requests
import re

In [5]:
@tool
def get_word_length(word: str) -> int: 
    """Returns the length of a word."""
    return len(word)

In [6]:
@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b

In [7]:
@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    
    response = requests.get(news_url)           # HTTP GET 요청 보내기

    if response.status_code == 200:             # 요청 성공
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)      # \n 이 2번이상 반복되는 것을 찾아서 \n으로 바꾼다.
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"

In [8]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    # 키워드로 뉴스 검색하는 도구 생성     
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [9]:
# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [10]:
# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [11]:
tools = [get_word_length, add_function, naver_news_crawl, search_keyword]

In [12]:
# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

In [16]:
# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [17]:
# Agent 실행
result = agent_executor.invoke({"input": "How many letters in the word `LEEINHWAN`?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'LEEINHWAN'}`


9The word "LEEINHWAN" has 9 letters.

> Finished chain.


In [18]:
print(result["output"])

The word "LEEINHWAN" has 9 letters.


In [21]:
result = agent_executor.invoke({"input": "대한민국 계엄령에대해서 네이버 뉴스에서 찾아줘"})



> Entering new AgentExecutor chain...

Invoking: `search_keyword` with `{'query': '대한민국 계엄령'}`


대한민국 계엄령
[{'url': 'https://news.google.com/rss/articles/CBMiW0FVX3lxTE9nUDFxLTk5N2Z4cWV2ZUpVejE4ZG5aY1VKSjNIRmdmVDk1RHp3YXVHdTNwbGNfa0g5SXlISk9NVUUxTjVPbWdDMHNhZUpaYTFMaVQtUEVVbXpmQjDSAV5BVV95cUxQUERleWdRM1dqTmtFejVLN3N2ZkdvaEdnelA4amtySjljdFN4TllISGhKVFU0b3M5aDVKUVRHOWZ6VE52TG5RQnJET1lpdWpjYkd5VnBxNTllLXFlN2xR?oc=5', 'content': '북한, 尹계엄령·탄핵 사태 첫 보도…"괴뢰한국 땅 아비규환" - 연합뉴스'}, {'url': 'https://news.google.com/rss/articles/CBMiXEFVX3lxTE5BcDBjMEc4Mkc4dG0zVzgxU1J0dDh2MFBRLTh2b2Jndl9RaEtWUVBwWXVwNmlrTjlBQ2tOWE1oN0NxamVWUjJCRmgzMENUUlRPdXl3bFdONjNUOXJj0gFiQVVfeXFMTlNZWENPNmNRalJaclBPR1N1ak16M1JxVll1Rm1Yem1OWDdpSUVIR3ZPcWhkOTFGNk1oRmtXMmctN2wyWlU1dExSaEdNTVh4aE9MTnhyVUpzMVR5emhQU1MtYkE?oc=5', 'content': "두 번째 담화 낸 윤석열 대통령 '탄핵하든 수사하든 맞서겠다' - BBC.com"}, {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE5YWHpjYUltRnBIZ1BiUzN6SDQyZlRFZFhLb0Y4WS02MmRyMDFWVGZ2MTd3WlpOak1GWmxmWlVEdlgtTW82MW44Vk

In [22]:
print(result["output"])

다음은 대한민국 계엄령에 대한 최근 뉴스 기사들입니다:

1. [북한, 尹계엄령·탄핵 사태 첫 보도…\"괴뢰한국 땅 아비규환\" - 연합뉴스](https://news.google.com/rss/articles/CBMiW0FVX3lxTE9nUDFxLTk5N2Z4cWV2ZUpVejE4ZG5aY1VKSjNIRmdmVDk1RHp3YXVHdTNwbGNfa0g5SXlISk9NVUUxTjVPbWdDMHNhZUpaYTFMaVQtUEVVbXpmQjDSAV5BVV95cUxQUERleWdRM1dqTmtFejVLN3N2ZkdvaEdnelA4amtySjljdFN4TllISGhKVFU0b3M5aDVKUVRHOWZ6VE52TG5RQnJET1lpdWpjYkd5VnBxNTllLXFlN2xR?oc=5)

2. [두 번째 담화 낸 윤석열 대통령 '탄핵하든 수사하든 맞서겠다' - BBC.com](https://news.google.com/rss/articles/CBMiXEFVX3lxTE5BcDBjMEc4Mkc4dG0zVzgxU1J0dDh2MFBRLTh2b2Jndl9RaEtWUVBwWXVwNmlrTjlBQ2tOWE1oN0NxamVWUjJCRmgzMENUUlRPdXl3bFdONjNUOXJj0gFiQVVfeXFMTlNZWENPNmNRalJaclBPR1N1ak16M1JxVll1Rm1Yem1OWDdpSUVIR3ZPcWhkOTFGNk1oRmtXMmctN2wyWlU1dExSaEdNTVh4aE9MTnhyVUpzMVR5emhQU1MtYkE?oc=5)

3. [김어준, 계엄 해제 후에도 36시간 은신…\"죽는 줄 알았다\" - 한국경제](https://news.google.com/rss/articles/CBMiWkFVX3lxTE5YWHpjYUltRnBIZ1BiUzN6SDQyZlRFZFhLb0Y4WS02MmRyMDFWVGZ2MTd3WlpOak1GWmxmWlVEdlgtTW82MW44Vk9iMW9scWNYcmlaNmk2QnFzUdIBVEFVX3lxTFAyZ0Q1d2t5NFpjRXg3cE56VXlEampHaVhIcDRT

In [23]:
# Agent 실행
result = agent_executor.invoke({"input": "114.5 + 121.2 의 계산 결과는?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7114.5 + 121.2의 계산 결과는 235.7입니다.

> Finished chain.
114.5 + 121.2의 계산 결과는 235.7입니다.


In [24]:
# Agent 실행
result = agent_executor.invoke(
    {"input": "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"}
)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 235.7, 'b': 34.2}`


269.9
Invoking: `add_function` with `{'a': 269.9, 'b': 110.1}`


380.0114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

> Finished chain.


In [25]:
result = agent_executor.invoke(
    {
        "input": "뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
    }
)



> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}`


[미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국

In [26]:
print(result["output"])

기사 요약:

9월 10일(현지시간) 미국의 소비자물가지수(CPI) 발표가 예정되어 있으며, 고용 지표가 양호한 가운데 물가 지표에 주목하고 있다. 9월 비농업 고용은 25만4천명 증가하여 시장 예상치를 크게 상회했으며, 실업률은 4.1%로 하락했다. 9월 헤드라인 CPI는 8월 대비 2.3% 증가할 것으로 예상되며, 근원 CPI는 3.2% 수준을 유지할 것으로 보인다.

11일에는 제조업 물가지수(PPI) 발표가 예정되어 있으며, S&P 500 기업의 3분기 실적 발표도 시작된다. 평균 이익 증가율은 4.6%로 예상되지만 증가폭은 둔화될 것으로 보인다. 한국은 FTSE 러셀의 시장접근성 등급이 상향 조정될 가능성이 있어 주목받고 있다.
